# Суммаризация текста

Большие языковые модели отлично подходят для генерации краткого содержания различных документов: PDF-файлов, страниц Notion, HTML-документов и других.

В контексте [генерации с RAG](/docs/tutorials/rag) суммаризация текста текста помогает выделить информацию из большого количества извлеченных документов, чтобы предоставить контекст для LLM.

В этом разделе вы узнаете как использовать LLM для суммаризации содержимого нескольких документов.

<Image img={require('../../static/img/summarization_use_case_1.png')} alt="Описание изображения" />

В разделе вы найдете информацию об:

- использовании [языковых моделей](/docs/concepts/#chat-models).

- использовании [загрузчиков документов](/docs/concepts/#document-loaders). В частности [WebBaseLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.web_base.WebBaseLoader.html) для загрузки содержимого HTML-страницы.

- трех способах суммаризации или комбинирования документов:

  - [Stuff](/docs/tutorials/summarization#stuff) — простое объединение документов в один промпт;
  - [Map-reduce](/docs/tutorials/summarization#map-reduce) — разделение документов на группы, суммаризация документов и генерация краткого содержания результатов суммаризации;
  - [Refine](/docs/tutorials/summarization#refine) — обновляет текущего результата суммаризации, по мере последовательной обработки документов.


## Подготовка

### Jupyter

Как и многие другие руководства в документации GigaChain это руководство основано на [блокноте Jupyter](https://jupyter.org/).
Блокноты предоставляют интерактивность, которая значительно упрощает изучение работы с LLM.

Об установке Jupyter читайте в [официальной документации](https://jupyter.org/install).

### Установка Зависимостей

В примере используются модели GigaChat (чат-модель и эмбеддинги), а также векторное хранилище Chroma, но в своем приложении вы можете использовать любой из доступных компонентов.

Для работы с примером нужно установить пакеты:

```python
%pip install --upgrade --quiet  gigachain langchainhub chromadb bs4
```

Подробнее об установке — в разделе [Установка GigaChain](/docs/how_to/installation)

<!--
### LangSmith

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls.
As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent.
The best way to do this is with [LangSmith](https://smith.langchain.com).

After you sign up at the link above, make sure to set your environment variables to start logging traces:

```shell
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="..."
```

Or, if in a notebook, you can set them with:

```python
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
```
-->

## Обзор

Главным вопросом вопрос при создании суммаризатора заключается в том, как передать документы в контекстное окно LLM.
Для решения этой задачи существует три распространенных подхода:

- `Stuff`. Просто поместите все свои документы в один промпт. Это самый простой подход. Для реализации этого способа используется конструктор `create_stuff_documents_chain`. Подробнее — в разделе [Встроенные цепочки](/docs/tutorials/rag#built-in-chains).

- `Map-reduce`. Сначала суммируйте каждый документ по отдельности на этапе *map*, а затем создайте краткое содержание итогом суммаризации на этапе о *reduce*. Для реализации этого способа используется класс  [`MapReduceDocumentsChain`](https://api.python.langchain.com/en/latest/chains/langchain.chains.combine_documents.map_reduce.MapReduceDocumentsChain.html).

- `Refine`. Обновляйте текущий результат суммаризации, последовательно обрабатывая документы.

<Image img={require('../../static/img/summarization_use_case_2.png')} alt="Описание изображения" />

## Быстрый старт

Любую из этих цепочек можно обернуть в один объект: `load_summarize_chain`.
Это позволит получить общее представление об итоговом решении.

Предположим, вы хотите получить краткое содержание записи в блоге.
Это можно сделать в несколько строк кода.

Установите переменные окружения и пакеты:

In [ ]:
%pip install --upgrade --quiet  langchain-openai tiktoken chromadb langchain

# Установите переменную окружения OPENAI_API_KEY или загрузите из файла .env
# import dotenv

# dotenv.load_dotenv()

Вы можете использовать `chain_type="stuff"`, особенно при работе с моделями с большим контекстным окном:

* 128k токенов OpenAI `gpt-4-turbo-2024-04-09` 
* 200k токенов Anthropic `claude-3-sonnet-20240229`

Вы также можете использовать `chain_type="map_reduce"` или `chain_type="refine"`.

Загрузите запись из блога с помощью загрузчика [WebBaseLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.web_base.WebBaseLoader.html):

In [2]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "True"

In [1]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-1106")
chain = load_summarize_chain(llm, chain_type="stuff")

result = chain.invoke(docs)

print(result["output_text"])

The article discusses the concept of LLM-powered autonomous agents, with a focus on the components of planning, memory, and tool use. It includes case studies and proof-of-concept examples, as well as challenges and references to related research. The author emphasizes the potential of LLMs in creating powerful problem-solving agents, while also highlighting limitations such as finite context length and reliability of natural language interfaces.


## Первый способ — Stuff {#stuff}

При работе `load_summarize_chain` с типом `chain_type="stuff"`  используется класс [StuffDocumentsChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.combine_documents.stuff.StuffDocumentsChain.html#langchain.chains.combine_documents.stuff.StuffDocumentsChain).

Цепочка возьмет список документов, вставит их все в один промпт и передаст этот промпт LLM:

In [4]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate

# Define prompt
prompt_template = """Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

docs = loader.load()
print(stuff_chain.invoke(docs)["output_text"])

The article discusses the concept of building autonomous agents powered by large language models (LLMs). It explores the components of such agents, including planning, memory, and tool use. The article provides case studies and examples of proof-of-concept demos, highlighting the challenges and limitations of LLM-powered agents. It also includes references to related research papers and projects.


В примере выше получается такой же результат как и при использовании `load_summarize_chain`.

:::tip

* Вы можете легко изменять промпт.
* С помощью параметра `llm` вы можете легко попробовать разные LLM, например, [Claude](/docs/integrations/chat/anthropic).

:::

## Второй способ — Map-Reduce {#map-reduce}

В этом способе вам сначала нужно с помощью `LLMChain` сопоставить каждый документ с результатами его суммаризации.
После чего использовать `ReduceDocumentsChain`, чтобы объединить все результаты в одно общее резюме.

Задайте цепочку LLMChain, которая будет сопоставлять каждый документ со своей суммаризацией:

In [5]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain_text_splitters import CharacterTextSplitter

llm = ChatOpenAI(temperature=0)

# Map
map_template = """The following is a set of documents
{docs}
Based on this list of docs, please identify the main themes 
Helpful Answer:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

<!--
Мы также можем использовать Prompt Hub для хранения и получения промптов.


Это будет работать с вашим [ключом API LangSmith](https://docs.smith.langchain.com/).

Например, см. пример промпта map [здесь](https://smith.langchain.com/hub/rlm/map-prompt).
-->

In [6]:
from langchain import hub

map_prompt = hub.pull("rlm/map-prompt")
map_chain = LLMChain(llm=llm, prompt=map_prompt)

Цепочка `ReduceDocumentsChain` обрабатывает результаты сопоставления документов и их краткого содержания до одного вывода.
Она оборачивает стандартную цепочку `CombineDocumentsChain` (например, `StuffDocumentsChain`) и добавляет возможность свернуть документы перед передачей их в `CombineDocumentsChain`, если их суммарный размер превышает `token_max`.
В этом примере вы сможете переиспользовать цепочку объединения документов `ReduceDocumentsChain`, чтобы также сворачивать их.

Таким образом, если суммарное количество токенов в сопоставленных документах превышает 4000 токенов, вы будете рекурсивно передавать документы группами не более 4000 токенов в цепочку `StuffDocumentsChain` для создания для создания суммарзации каждой из групп.
И как только результаты суммаризации каждой из групп в сумме будут меньше 4000 токенов, вы передадите их все в последний раз в `StuffDocumentsChain` для создания итогового краткого содержания.

In [7]:
# Reduce
reduce_template = """The following is set of summaries:
{docs}
Take these and distill it into a final, consolidated summary of the main themes. 
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

In [8]:
# Обратите внимание, что мы также можем получить это из Prompt Hub, как указано выше
reduce_prompt = hub.pull("rlm/reduce-prompt")

In [9]:
reduce_prompt

ChatPromptTemplate(input_variables=['docs'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'map-prompt', 'lc_hub_commit_hash': 'de4fba345f211a462584fc25b7077e69c1ba6cdcf4e21b7ec9abe457ddb16c87'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['docs'], template='The following is a set of documents:\n{docs}\nBased on this list of docs, please identify the main themes \nHelpful Answer:'))])

In [11]:
# Запуск цепочки
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Получение на вход списка документов
# объединение их в одну строку
# передача строки в цепочку LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# Объединение и итеративное уменьшение сопоставленных документов
reduce_documents_chain = ReduceDocumentsChain(
    # Итоговая цепочка, которая
    combine_documents_chain=combine_documents_chain,
    # вызывается если документы превышают контекст `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # Колчичество токенов, которое ограничивает размер группы документов.
    token_max=4000,
)

Объединение сопоставленных элементов и сокращение до одной цепочки.

In [12]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

Created a chunk of size 1003, which is longer than the specified 1000


In [16]:
result = map_reduce_chain.invoke(split_docs)

print(result["output_text"])

The main themes identified in the list of documents provided are related to large language models (LLMs), autonomous agents, prompting, steering language models, natural language processing (NLP), the use of tools to augment language models, reinforcement learning, reasoning, acting, self-reflection, and the integration of language models with external knowledge sources.


<!--
Если мы следуем по [Langsmith Trace](https://smith.langchain.com/public/3a1a6d51-68e5-4805-8d90-78920ce60a51/r), мы можем увидеть индивидуальные суммирования LLM, включая [финальный вызов](https://smith.langchain.com/public/69482813-f0b7-46b0-a99f-86d56fc9644a/r), который суммирует все резюме.

### Углубимся

**Кастомизация**

* Как показано выше, вы можете настраивать LLM и промпты для этапов map и reduce.

**Практическое применение**

* См. [этот блог-пост](https://blog.langchain.dev/llms-to-improve-documentation/) с кейс-стади по анализу взаимодействий пользователей (вопросы о документации LangChain)!  
* Блог-пост и связанный [репозиторий](https://github.com/mendableai/QA_clustering) также вводят кластеризацию как способ суммирования.
* Это открывает еще один путь, помимо подходов `stuff` или `map-reduce`, который стоит рассмотреть.
-->

<Image img={require('../../static/img/summarization_use_case_3.png')} alt="Описание изображения" />

## Третий способ — Refine {#refine}

Цепочка [RefineDocumentsChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.combine_documents.refine.RefineDocumentsChain.html) похожа на способ map-reduce:

Цепочка `refine documents` создает ответ, перебирая входные документы и итеративно обновляя созданный ответ.
Для каждого документа она передает входные данные, текущий документ и последнее промежуточный ответ в цепочку LLM для получения нового ответа.

Для этого укажите тип `chain_type="refine"`.

In [21]:
chain = load_summarize_chain(llm, chain_type="refine")
result = chain.invoke(split_docs)

print(result["output_text"])

The existing summary provides detailed instructions for implementing a project's architecture through code, focusing on creating core classes, functions, and methods in different files following best practices for the chosen language and framework. Assumptions about the model, view, and controller components are also outlined. The additional context highlights challenges in long-term planning and task decomposition, as well as the reliability issues with natural language interfaces in LLM-powered autonomous agents. These insights shed light on the limitations and potential pitfalls of using LLMs in agent systems, with references to recent research on LLM-powered autonomous agents and related technologies.


<!--
Following the [Langsmith trace](https://smith.langchain.com/public/38017fa7-b190-4635-992c-e8554227a4bb/r), we can see the summaries iteratively updated with new information.
-->

Вы также можете передать промпт и вернуть промежуточные шаги.

In [22]:
prompt_template = """Write a concise summary of the following:
{text}
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary in Italian"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate.from_template(refine_template)
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)
result = chain({"input_documents": split_docs}, return_only_outputs=True)

/Users/chestercurme/repos/langchain/libs/core/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [14]:
prompt_template = """Write a concise summary of the following:
{text}
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary in Italian"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate.from_template(refine_template)
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)
result = chain.invoke({"input_documents": split_docs}, return_only_outputs=True)

In [15]:
print(result["output_text"])

Il presente articolo discute il concetto di costruire agenti autonomi utilizzando LLM (large language model) come controller principale. Esplora i diversi componenti di un sistema di agenti alimentato da LLM, tra cui la pianificazione, la memoria e l'uso degli strumenti. Dimostrazioni di concetto come AutoGPT mostrano il potenziale di LLM come risolutore generale di problemi. Approcci come Chain of Thought, Tree of Thoughts, LLM+P, ReAct e Reflexion consentono agli agenti autonomi di pianificare, riflettere su se stessi e migliorarsi iterativamente. Tuttavia, ci sono sfide da affrontare, come la limitata capacità di contesto che limita l'inclusione di informazioni storiche dettagliate e la difficoltà di pianificazione a lungo termine e decomposizione delle attività. Inoltre, l'affidabilità dell'interfaccia di linguaggio naturale tra LLM e componenti esterni come la memoria e gli strumenti è incerta, poiché i LLM possono commettere errori di formattazione e mostrare comportamenti ribell

In [16]:
print("\n\n".join(result["intermediate_steps"][:3]))

This article discusses the concept of building autonomous agents using LLM (large language model) as the core controller. The article explores the different components of an LLM-powered agent system, including planning, memory, and tool use. It also provides examples of proof-of-concept demos and highlights the potential of LLM as a general problem solver.

Questo articolo discute del concetto di costruire agenti autonomi utilizzando LLM (large language model) come controller principale. L'articolo esplora i diversi componenti di un sistema di agenti alimentato da LLM, inclusa la pianificazione, la memoria e l'uso degli strumenti. Vengono forniti anche esempi di dimostrazioni di proof-of-concept e si evidenzia il potenziale di LLM come risolutore generale di problemi. Inoltre, vengono presentati approcci come Chain of Thought, Tree of Thoughts, LLM+P, ReAct e Reflexion che consentono agli agenti autonomi di pianificare, riflettere su se stessi e migliorare iterativamente.

Questo artic

## Разделение и суммаризация в одной цепочке

Для удобства вы можете обернуть разделение текста длинного документа и его уммаризацию в одной цепочке `AnalyzeDocumentsChain`.

In [ ]:
from langchain.chains import AnalyzeDocumentChain

summarize_document_chain = AnalyzeDocumentChain(
    combine_docs_chain=chain, text_splitter=text_splitter
)
summarize_document_chain.invoke(docs[0].page_content)

## Смотрите также

- [Работа с загрузчиками документов](/docs/how_to/#document-loaders)
- [Работа с разделителями текста](/docs/how_to/#text-splitters)
- [Создание RAG-приложения](/docs/tutorials/rag/)
- [Поиск в чат-боте](/docs/how_to/chatbots_retrieval/)